## Creating MCP Server

In [1]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers together"""
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """Greet a user by name"""
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

Writing mcp_server.py


## Creating client to test locally

In [2]:
%%writefile my_mcp_client.py
import asyncio
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

Writing my_mcp_client.py


## Local testin

In [6]:
# python mcp_server.py in CLI
!python my_mcp_client.py

Available tools:
  - add_numbers: Add two numbers together
  - multiply_numbers: Multiply two numbers together
  - greet_user: Greet a user by name


## Setting up Amazon Cognito for Authentication

In [12]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

# utils_dir = os.path.join(current_dir, '..')
# utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
# sys.path.insert(0, utils_dir)
sys.path.insert(0, current_dir)
print("sys.path[0]:", sys.path[0])

from utils import setup_cognito_user_pool
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")
print(f"User Pool ID: {cognito_config.get('user_pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

sys.path[0]: /Users/ykhvainitski/repos/bedrock-strands
Setting up Amazon Cognito user pool...
Pool id: us-east-1_awjGJtQxs
Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_awjGJtQxs/.well-known/openid-configuration
Client ID: 6en4llrv22q7cdsqnat97d4175
Bearer Token: eyJraWQiOiI2R2M5WjVaTzJ6Q0pFTlp1UiszSjNRUFI2YWFlQk5OZUJLMzMwY0RZV2xVPSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiI1NDY4MTRjOC1jMGIxLTcwOTYtNWM5Yy04YjZjMjIwYjUzYWYiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV9hd2pHSnRReHMiLCJjbGllbnRfaWQiOiI2ZW40bGxydjIycTdjZHNxbmF0OTdkNDE3NSIsIm9yaWdpbl9qdGkiOiIwMjM5NGU0Yi1hNTg3LTQxNmItYTBhNi0xMDUwYTZmNWFiYjEiLCJldmVudF9pZCI6ImI4MjFkOGYyLWNiYjktNDcwNi05NjE0LTlkYWE5YmY5MmIwZSIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NTYzMDk1MjYsImV4cCI6MTc1NjMxMzEyNiwiaWF0IjoxNzU2MzA5NTI2LCJqdGkiOiIwMDVkNmRkOS03YzE5LTQwMWMtYjRjYy1kYzJlNzgyYmNkMTciLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.E7ObTVHF8cHJkqrw35cug

## Configuring AgentCore Runtime Deployment

In [13]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['mcp_server.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name="mcp_server_agentcore"
)
print("Configuration completed ✓")

Entrypoint parsed: file=/Users/ykhvainitski/repos/bedrock-strands/ac_runtime_tutorial_mcp/mcp_server.py, bedrock_agentcore_name=mcp_server
Configuring BedrockAgentCore agent: mcp_server_agentcore


Using AWS region: us-east-1
All required files found ✓
Configuring AgentCore Runtime...


Generated .dockerignore
Generated Dockerfile: /Users/ykhvainitski/repos/bedrock-strands/ac_runtime_tutorial_mcp/Dockerfile
Generated .dockerignore: /Users/ykhvainitski/repos/bedrock-strands/ac_runtime_tutorial_mcp/.dockerignore
Setting 'mcp_server_agentcore' as default agent
Bedrock AgentCore configured: /Users/ykhvainitski/repos/bedrock-strands/ac_runtime_tutorial_mcp/.bedrock_agentcore.yaml


Configuration completed ✓


## Launching MCP Server to AgentCore Runtime

In [14]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'mcp_server_agentcore' to account 538213298977 (us-east-1)
Starting CodeBuild ARM64 deployment for agent 'mcp_server_agentcore' to account 538213298977 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: mcp_server_agentcore


Launching MCP server to AgentCore Runtime...
This may take several minutes...
Repository doesn't exist, creating new ECR repository: bedrock-agentcore-mcp_server_agentcore


✅ ECR repository available: 538213298977.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-mcp_server_agentcore
Getting or creating execution role for agent: mcp_server_agentcore
Using AWS region: us-east-1, account ID: 538213298977
Role name: AmazonBedrockAgentCoreSDKRuntime-us-east-1-7f3ae149b4
Role doesn't exist, creating new execution role: AmazonBedrockAgentCoreSDKRuntime-us-east-1-7f3ae149b4
Starting execution role creation process for agent: mcp_server_agentcore
✓ Role creating: AmazonBedrockAgentCoreSDKRuntime-us-east-1-7f3ae149b4
Creating IAM role: AmazonBedrockAgentCoreSDKRuntime-us-east-1-7f3ae149b4
✓ Role created: arn:aws:iam::538213298977:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-7f3ae149b4
✓ Execution policy attached: BedrockAgentCoreRuntimeExecutionPolicy-mcp_server_agentcore
Role creation complete and ready for use with Bedrock AgentCore
✅ Execution role available: arn:aws:iam::538213298977:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-7f3ae149b4
Preparing CodeB

Launch completed ✓
Agent ARN: arn:aws:bedrock-agentcore:us-east-1:538213298977:runtime/mcp_server_agentcore-YeR1XhFl9z
Agent ID: mcp_server_agentcore-YeR1XhFl9z


In [15]:
import time

print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

Checking AgentCore Runtime status...


Retrieved Bedrock AgentCore status for: mcp_server_agentcore


Initial status: READY
✓ AgentCore Runtime is READY!
Final status: READY


## Storing Configuration for Remote Access

Before we can invoke our deployed MCP server, let's store the Agent ARN and Cognito configuration in AWS Systems Manager Parameter Store and AWS Secrets Manager for easy retrieval:

In [16]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='mcp_server/cognito/credentials',
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='mcp_server/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

✓ Cognito credentials stored in Secrets Manager
✓ Agent ARN stored in Parameter Store

Configuration stored successfully!
Agent ARN: arn:aws:bedrock-agentcore:us-east-1:538213298977:runtime/mcp_server_agentcore-YeR1XhFl9z


## Creating Remote Testing Client

In [17]:
%%writefile my_mcp_client_remote.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    if not agent_arn or not bearer_token:
        print("Error: AGENT_ARN or BEARER_TOKEN not retrieved properly")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")
    print("Headers configured ✓")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()
                
                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

Writing my_mcp_client_remote.py


### Testing

In [18]:
print("Testing deployed MCP server...")
print("=" * 50)
!python my_mcp_client_remote.py

Testing deployed MCP server...
Using AWS region: us-east-1
Retrieved Agent ARN: arn:aws:bedrock-agentcore:us-east-1:538213298977:runtime/mcp_server_agentcore-YeR1XhFl9z
✓ Retrieved bearer token from Secrets Manager

Connecting to: https://bedrock-agentcore.us-east-1.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aus-east-1%3A538213298977%3Aruntime%2Fmcp_server_agentcore-YeR1XhFl9z/invocations?qualifier=DEFAULT
Headers configured ✓

🔄 Initializing MCP session...
✓ MCP session initialized

🔄 Listing available tools...

📋 Available MCP Tools:
🔧 add_numbers
   Description: Add two numbers together
   Parameters: ['a', 'b']

🔧 multiply_numbers
   Description: Multiply two numbers together
   Parameters: ['a', 'b']

🔧 greet_user
   Description: Greet a user by name
   Parameters: ['name']

✅ Successfully connected to MCP server!
Found 3 tools available.


## Invoking MCP Tools Remotely

In [19]:
%%writefile invoke_mcp_tools.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")
                
                print("\n🧪 Testing MCP Tools:")
                print("=" * 50)
                
                try:
                    print("\n➕ Testing add_numbers(5, 3)...")
                    add_result = await session.call_tool(
                        name="add_numbers",
                        arguments={"a": 5, "b": 3}
                    )
                    print(f"   Result: {add_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n✖️  Testing multiply_numbers(4, 7)...")
                    multiply_result = await session.call_tool(
                        name="multiply_numbers",
                        arguments={"a": 4, "b": 7}
                    )
                    print(f"   Result: {multiply_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n👋 Testing greet_user('Alice')...")
                    greet_result = await session.call_tool(
                        name="greet_user",
                        arguments={"name": "Alice"}
                    )
                    print(f"   Result: {greet_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                print("\n✅ MCP tool testing completed!")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

Writing invoke_mcp_tools.py


### Testing

In [20]:
print("Testing MCP tool invocation...")
print("=" * 50)
!python invoke_mcp_tools.py

Testing MCP tool invocation...
Using AWS region: us-east-1
Retrieved Agent ARN: arn:aws:bedrock-agentcore:us-east-1:538213298977:runtime/mcp_server_agentcore-YeR1XhFl9z
✓ Retrieved bearer token from Secrets Manager

Connecting to: https://bedrock-agentcore.us-east-1.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aus-east-1%3A538213298977%3Aruntime%2Fmcp_server_agentcore-YeR1XhFl9z/invocations?qualifier=DEFAULT

🔄 Initializing MCP session...
✓ MCP session initialized

🔄 Listing available tools...

📋 Available MCP Tools:
🔧 add_numbers: Add two numbers together
🔧 multiply_numbers: Multiply two numbers together
🔧 greet_user: Greet a user by name

🧪 Testing MCP Tools:

➕ Testing add_numbers(5, 3)...
   Result: 8

✖️  Testing multiply_numbers(4, 7)...
   Result: 28

👋 Testing greet_user('Alice')...
   Result: Hello, Alice! Nice to meet you.

✅ MCP tool testing completed!


## Cleanup

In [21]:
import boto3

print("🗑️  Starting cleanup process...")

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)
ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    print("Deleting AgentCore Runtime...")
    runtime_delete_response = agentcore_control_client.delete_agent_runtime(
        agentRuntimeId=launch_result.agent_id,
    )
    print("✓ AgentCore Runtime deletion initiated")

    print("Deleting ECR repository...")
    ecr_repo_name = launch_result.ecr_uri.split('/')[1]
    ecr_client.delete_repository(
        repositoryName=ecr_repo_name,
        force=True
    )
    print("✓ ECR repository deleted")

    try:
        ssm_client.delete_parameter(Name='/mcp_server/runtime/agent_arn')
        print("✓ Parameter Store parameter deleted")
    except ssm_client.exceptions.ParameterNotFound:
        print("ℹ️  Parameter Store parameter not found")
    try:
        secrets_client.delete_secret(
            SecretId='mcp_server/cognito/credentials',
            ForceDeleteWithoutRecovery=True
        )
        print("✓ Secrets Manager secret deleted")
    except secrets_client.exceptions.ResourceNotFoundException:
        print("ℹ️  Secrets Manager secret not found")

    print("\n✅ Cleanup completed successfully!")
    
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")

🗑️  Starting cleanup process...
Deleting AgentCore Runtime...
✓ AgentCore Runtime deletion initiated
Deleting ECR repository...
✓ ECR repository deleted
✓ Parameter Store parameter deleted
✓ Secrets Manager secret deleted

✅ Cleanup completed successfully!
